In [61]:
import requests 
from bs4 import BeautifulSoup as soup
import re
import numpy as np
import pandas as pd

from pymongo import MongoClient
user_ids=pd.read_csv('./userid.csv');


In [62]:
# Create connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['research']
collection = db['researchers']

In [64]:
url_base = 'https://researchers.a-star.edu.sg/userprofile.aspx?userid='
for id in user_ids.userid:
    try:
        url=url_base+str(id)
        html_page = requests.get(url)
        soup_data = soup(html_page.content,'html.parser')
        container = soup_data.find('div',{'id':'dnn_ctr429_UserProfile_profile'})
        
        if soup_data.find('span',{'id':'dnn_ctr429_UserProfile_lblUserProfile'}).find('b'):
            user_name = soup_data.find('span',{'id':'dnn_ctr429_UserProfile_lblUserProfile'})
            user_name = user_name.find('b').text
            name=re.sub('- Profile','',user_name).encode("utf-8")
        else :
            name="null"
            
        if container.find('span',{'id':'dnn_ctr429_UserProfile_lbluserid'}):
            user_id = container.find('span',{'id':'dnn_ctr429_UserProfile_lbluserid'}).text.encode("utf-8")
        else :
            user_id="null"
            
        if container.find('a',{'id':'dnn_ctr429_UserProfile_lblEmail2'}):
            email = container.find('a',{'id':'dnn_ctr429_UserProfile_lblEmail2'}).text.encode("utf-8")
        else :
            email="null"
            
        if container.find('span',{'id':'dnn_ctr429_UserProfile_lblOrganisation2'}):
            organisation = container.find('span',{'id':'dnn_ctr429_UserProfile_lblOrganisation2'}).text.encode("utf-8")
        else :
            organisation="null"
            
        if container.find('span',{'id':'dnn_ctr429_UserProfile_lblTel2'}):
            tel_no = container.find('span',{'id':'dnn_ctr429_UserProfile_lblTel2'}).text.encode("utf-8")
        else :
            tel_no="null"
            
        if container.find('a',{'id':'dnn_ctr429_UserProfile_hpkUrl'}):
            person_url = container.find('a',{'id':'dnn_ctr429_UserProfile_hpkUrl'}).text.encode("utf-8")
        else :
            person_url="null"
            
        if container.find('table',{'id':'dnn_ctr429_UserProfile_dlKeyword'}) and \
            container.find('table',{'id':'dnn_ctr429_UserProfile_dlKeyword'}).findAll('tr'):
            keywords=[]
            keywords_table = container.find('table',{'id':'dnn_ctr429_UserProfile_dlKeyword'})
            for x in range(0,len(keywords_table.findAll('tr'))):
                keywords.append(keywords_table.findAll('tr')[x].get_text().strip().encode("utf-8"))
        else :
            keywords=[]
            
        if container.find('span',{'id':'dnn_ctr429_UserProfile_lstAreaofResearch'}) and \
            container.find('span',{'id':'dnn_ctr429_UserProfile_lstAreaofResearch'}).findAll('span'):
            research_areas=[]
            research_area = container.find('span',{'id':'dnn_ctr429_UserProfile_lstAreaofResearch'})
            research_area = research_area.findAll('span')
            for x in range(0,len(research_area)):
                if research_area[x].get_text().strip().encode("utf-8") > '':
                    research_areas.append(research_area[x].get_text().strip().encode("utf-8"))
        else :
            research_areas=[]
            
        if container.find('div',{'id':'dnn_ctr429_UserProfile_tblEducation'}) and \
        container.find('div',{'id':'dnn_ctr429_UserProfile_tblEducation'}).findAll('table'):
            eduMainTable = container.find('div',{'id':'dnn_ctr429_UserProfile_tblEducation'}).table
            eduSchools = eduMainTable.findAll('table')
            schools=[]
            for i,table in enumerate(eduSchools):
                school=""
                for i,entry in enumerate(table.findAll('td')):
                    school=school+entry.get_text().strip().encode("utf-8")+","
                schools.append(school)
        else :
            schools=[]
            
        if container.find('table',{'id':'tblempHistory'}) :
            EmpMainTable = container.find('table',{'id':'tblempHistory'})
            EmpHistory = EmpMainTable.findAll('table')
            companies=[]
            for i,table in enumerate(EmpHistory):
                if i==0 : continue
                company={}
                company['designation']=table.findAll('td')[0].get_text().strip().encode("utf-8")
                company['duration']=table.findAll('td')[1].get_text().strip().encode("utf-8")
                company['organization']=table.findAll('td')[2].get_text().strip().encode("utf-8")
                company['country']=table.findAll('td')[3].get_text().strip().encode("utf-8")
                companies.append(company)      
        else :
            companies=[]
            
        if container.find('table',{'id':'tblPublication2'}) :
            PubMainTable = container.find('table',{'id':'tblPublication2'})
            pubTables = PubMainTable.findAll('table')
            publications=[]
            for i,table in enumerate(pubTables):
                if i==0 : continue
                publication={}
                publication['author']=table.findAll('td')[0].get_text().strip().encode("utf-8")
                publication['title']=table.findAll('td')[1].get_text().strip().encode("utf-8")
                publication['journal']=table.findAll('td')[2].get_text().strip().encode("utf-8")
                publication['year']=table.findAll('td')[3].get_text().strip().encode("utf-8")
                publication['vol:page']=table.findAll('td')[4].get_text().strip().encode("utf-8")
                publications.append(publication)
        else :
            publications=[]
        
        if container.find('table',{'id':'tblPatents2'}):
            patMainTable = container.find('table',{'id':'tblPatents2'})
            patTables = patMainTable.findAll('table')
            patents=[]
            for i,table in enumerate(patTables):
                if i==0 : continue
                patent={}
                patent['title']=table.findAll('td')[0].get_text().strip().encode("utf-8")
                patent['number']=table.findAll('td')[1].get_text().strip().encode("utf-8")
                patent['year_awarded']=table.findAll('td')[2].get_text().strip().encode("utf-8")
                patent['country']=table.findAll('td')[3].get_text().strip().encode("utf-8")
                patents.append(patent)
        else :
            patents=[]
            
        if container.find('table',{'id':'tblAwards2'}):
            awardsMainTable = container.find('table',{'id':'tblAwards2'})
            awardsTables = awardsMainTable.findAll('table')
            awards=[]
            for i,table in enumerate(awardsTables):
                if i==0 : continue
                award={}
                award['description']=table.findAll('td')[0].get_text().strip().encode("utf-8")
                award['year']=table.findAll('td')[1].get_text().strip().encode("utf-8")
                award['organization_country']=table.findAll('td')[2].get_text().strip().encode("utf-8")
                awards.append(award)
        else :
            awards=[]
            
        if container.find('div',{'id':'dnn_ctr429_UserProfile_tblEditorship'}):
            editMainTable = container.find('div',{'id':'dnn_ctr429_UserProfile_tblEditorship'})
            editTables = editMainTable.findAll('table')
            edits=[]
            for i,table in enumerate(editTables):
                if i==0 or i==1 or i==2: continue
                edit={}
                edit['description']=table.findAll('td')[0].get_text().strip().encode("utf-8")
                edit['from_year']=table.findAll('td')[1].get_text().strip().encode("utf-8")
                edit['to_year']=table.findAll('td')[2].get_text().strip().encode("utf-8")
                edit['organization_country']=table.findAll('td')[3].get_text().strip().encode("utf-8")
                edits.append(edit)
        else :
            edits=[]
            
        if container.find('div',{'id':'dnn_ctr429_UserProfile_tblGrant'}):
            grantMainTable = container.find('div',{'id':'dnn_ctr429_UserProfile_tblGrant'})
            grantTables = grantMainTable.findAll('table')
            edits=[]

            for i,table in enumerate(grantTables):
                if i==0 or i==1 or i==2: continue
                grant={}
                grant['project_title']=table.findAll('td')[0].get_text().strip().encode("utf-8")
                grants.append(grant)
        else :
            grants=[]
            
    except Exception as ex:
        print ex
        [name,user_id,email,organisation,tel_no,person_url,keywords, \
            research_areas,schools,companies,publications,patents,awards,edits,grants] = \
        ['null','null','null','null','null','null','null','null','null','null','null','null',\
        'null','null','null',]
    
    user={}
    user['name']=name
    user['user_id']=user_id
    user['organisation']=organisation
    user['tel_no']=tel_no
    user['person_url']=person_url
    user['keywords']=keywords
    user['research_areas']=research_areas
    user['schools']=schools
    user['companies']=companies
    user['publications']=publications
    user['patents']=patents
    user['awards']=awards
    user['edits']=edits
    user['grants']=grants
    user['email']=email
    collection.insert_one(user)
    print id



8
11
24
25
26
28
32
33
34
35
39
53
56
57
58
71
77
78
81
83
88
89
93
94
95
101
103
104
106
108
109
113
115
116
117
119
120
129
132
134
136
138
139
140
142
143
144
145
146
147
148
149
150
153
154
156
158
159
160
161
162
163
167
168
171
172
173
174
177
178
179
181
182
185
188
189
190
192
196
197
198
199
201
202
203
204
205
206
209
210
212
213
214
215
219
221
225
227
228
230
231
233
240
243
246
247
248
249
250
251
254
256
258
260
261
263
267
268
269
273
274
275
276
277
279
280
281
285
287
290
291
306
309
316
321
322
323
324
327
328
329
331
332
333
334
336
337
338
339
340
341
344
345
347
348
349
350
351
354
356
358
360
361
363
364
365
366
368
374
377
379
380
383
386
391
392
393
395
396
398
400
402
407
408
412
413
415
416
419
423
426
427
436
441
442
443
450
451
452
453
454
455
456
457
458
461
462
465
469
470
473
474
475
478
481
484
487
495
497
498
499
502
503
506
507
509
510
511
517
518
519
520
521
522
526
528
530
531
532
537
538
539
540
542
543
547
549
553
557
560
561
562
563
570
573
574
57

7573
7575
7580
7581
7582
7584
7586
7587
7589
7593
7595
7597
7598
7599
7601
7602
7603
7604
7606
7608
7610
7613
7614
7615
7616
7618
7619
7621
7622
7624
7629
7631
7634
7635
7636
7637
7638
7639
7640
7641
7649
7650
7651
7653
7654
7657
7658
7660
7662
7664
7665
7666
7668
7672
7673
7674
7676
7677
7678
7680
7681
7682
7683
7687
7688
7691
7692
7694
7697
7699
7700
7703
7707
7708
7711
7714
7716
7717
7718
7723
7724
7727
7731
7732
7733
7734
7736
7737
7739
7741
7743
7748
7750
7751
7754
7755
7756
7758
7764
7766
7769
7770
7771
7772
7773
7774
7775
7779
7780
7781
7787
7790
7792
7793
7794
7797
7798
7802
7803
7806
7809
7811
7812
7813
7814
7816
7817
7820
7821
7822
7824
7826
7827
7828
7829
7831
7834
7835
7837
7838
7840
7841
7843
7845
7847
7848
7849
7853
7856
7857
7860
7863
7866
7867
7869
7871
7872
7873
7874
7875
7877
7878
7879
7881
7882
7883
7886
7888
7891
7894
7895
7896
7897
7899
7900
7901
7906
7908
7910
7911
7914
7915
7916
7917
7918
7920
7921
7922
7923
7925
7926
7927
7929
7931
7934
7936
7938
7939
7940
7941


10048
10049
10050
10052
10053
10056
10057
10058
10059
10060
10061
10064
10066
10068
10069
10070
10072
10073
10074
10076
10077
10078
10079
10080
10081
10082
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10107
10108
10109
10110
10111
10113
10114
10115
10118
10119
10120
10121
10123
10124
10125
10126
10128
10129
10131
10132
10133
10134
10136
10138
10139
10140
10141
10142
10143


In [80]:
url_base = 'https://researchers.a-star.edu.sg/userprofile.aspx?userid=6932'
html_page = requests.get(url_base)
soup_data = soup(html_page.content,'html.parser')
container = soup_data.find('div',{'id':'dnn_ctr429_UserProfile_profile'})

In [81]:
e = 'dnn_ctr429_UserProfile_tblCollaborations'